In [ ]:
import re
import glob
import pandas as pd
import sweetviz
import random
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
import pycld2 as cld2

sys.path.append('../')

In [ ]:
import configparser
configParser = configparser.RawConfigParser()   
configFilePath = r'config.txt'
configParser.read(configFilePath)
csvpath = configParser.get('paths', 'csvpath')
outpath = configParser.get('paths', 'outpath')
outpath,csvpath

In [ ]:
from preprocessing.amharicSegmenter import AmharicSegmenter


In [ ]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [ ]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [ ]:
def remove_eng(data):
    return ' '.join(re.sub(u"[^ሀ-፼]", " ", data).split()).strip()
     

In [ ]:
def remove_emoji(news):
    news = emoji.demojize(news)
    re.sub('(:\S+)(@\w+)','', news)
    return news

In [ ]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [ ]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

In [11]:
outfile = outpath+"/am_all_sentences.txt" #tig_all_sentences.txt for tigregna
os.remove(outfile) if os.path.exists(outfile) else None

allnews = []
for news in glob.glob(csvpath+"/*.csv"):
    allnews.append(news)
    
# read each files and write to a file system 

with open (outfile,"a", encoding="utf-8") as all_sentences:
    lines_seen = set() # holds lines already seen
    for news in tqdm.tqdm(allnews,  position=0, leave=True):
        data = pd.read_csv(news, names=["id","url","date","title","text","html"] ,encoding="utf-8")
        data = data[data.text.duplicated()==False].reset_index()
        data.text = data.text.apply(lambda x: remove_url(x))
        #data.text = data.text.apply(lambda x: remove_emoji(x))
        #data.text = data.text.apply(lambda x: remove_eng(x))
        #for token in remove:
        #    data.text = data.text.apply(lambda x: x.replace(token,' '))  
        for text in data.text:
            for s in amseg.tokenize_sentence(text):
                isReliable, textBytesFound, details  = cld2.detect(s)
                if details[0][1] =='am' and s not in lines_seen: # am=amharic ti=tigrenya,                 
                    all_sentences.write(' '.join(t.text for t in amseg.amharic_tokenizer(s))+'\n')
                    lines_seen.add(s)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.50s/it]


In [ ]:
#for Orommigna
!pip install nltk


In [12]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
oro = pd.read_csv('2020-11-24-0-0-2.csv', names=["id","url","date","title","text"] ,encoding="utf-8")
first_row = oro.loc[0, "text"]
sent_tokenize(first_row)


['Akkuma fakkii Lii’oonaardoo Daavinchii kana ilaallee dinqisiifannu jaarraa 16ffaa keessa Xaaliyaanitti yeroo fakkiin kun kaafamu foolii ture suufuu otoo dandeenyee hoo?',
 '22 Sadaasa 2020\nViidiyoon kabaja “waadaa galuu” Imbaasii Ruwaandaa Landan keessatti yemmuu gaggeeffamuu agarsiisu ba’e, biyyaa Baha Afrikaa qeeqtootaan “Kooriyaa Kaabaa” haaraa jedhamte irratti himannaa abba irrummaa jiru guddiseera.',
 '19 Sadaasa 2020\nJoy Baayiden waggoota 40 oliif nama siyaasa Ameerikaa keessatti haalaan beekamudha.',
 "Bu'aa bahiin jireenya isaanii maal fakkaata?",
 '7 Sadaasa 2020\nBaqataan Itoophiyaa dargaggummaan Ameerikaatti godaanuun, dhiyeenyatti lammummaa biyyattii argate, kutaa biyyaa Mineesotaa teessoo mana maree Ostin injifachuun seenaa galmeesse.',
 '11 Sadaasa 2020\nWaldhabdeen paartii naannoo Tigiraay bulchaa jiruu (TPLF)fi mootummaa giddu-galeessaa gidduu jiru sadarkaa walitti dhukaasuurra gaheera.',
 'Kanarra darbees Ministirri Muummee Abiy Ahimad raayyaan ittisa biyyaa tarkaa

In [23]:
#oro['text'].apply(nltk.sent_tokenize)